In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import webbrowser
import os

In [2]:
apps_df=pd.read_csv('Play Store Data.csv ')
reviews_df=pd.read_csv('User Reviews.csv')

In [3]:
apps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [4]:
reviews_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [5]:
apps_df=apps_df.dropna(subset=['Rating'])
for column in apps_df.columns:
    apps_df[column].fillna(apps_df[column].mode()[0],inplace=True)
apps_df.drop_duplicates(inplace=True)
apps_df=apps_df=apps_df[apps_df['Rating']<=5]
reviews_df.dropna(subset=["Translated_Review"],inplace=True)

In [6]:
reviews_df.dtypes

App                        object
Translated_Review          object
Sentiment                  object
Sentiment_Polarity        float64
Sentiment_Subjectivity    float64
dtype: object

In [7]:
apps_df['Installs']=apps_df['Installs'].str.replace(',','').str.replace('+','').astype(int)
apps_df['Price']=apps_df['Price'].str.replace('$','').astype(float)

In [8]:
apps_df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs            int32
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [9]:
apps_df['Type']

0        Free
1        Free
2        Free
3        Free
4        Free
         ... 
10834    Free
10836    Free
10837    Free
10839    Free
10840    Free
Name: Type, Length: 8892, dtype: object

In [10]:
merged_df=pd.merge(apps_df,reviews_df,on='App',how='inner')

In [11]:
merged_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,A kid's excessive ads. The types ads allowed a...,Negative,-0.250,1.000000
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,It bad >:(,Negative,-0.725,0.833333
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,like,Neutral,0.000,0.000000
3,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I love colors inspyering,Positive,0.500,0.600000
4,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I hate,Negative,-0.800,0.900000


In [12]:
merged_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,A kid's excessive ads. The types ads allowed a...,Negative,-0.250000,1.000000
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,It bad >:(,Negative,-0.725000,0.833333
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,like,Neutral,0.000000,0.000000
3,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I love colors inspyering,Positive,0.500000,0.600000
4,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I hate,Negative,-0.800000,0.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59119,A+ Gallery - Photos & Videos,PHOTOGRAPHY,4.5,223941,Varies with device,10000000,Free,0.0,Everyone,Photography,"August 6, 2018",Varies with device,Varies with device,Overall great app. Best gallery seen far,Positive,0.475000,0.512500
59120,A+ Gallery - Photos & Videos,PHOTOGRAPHY,4.5,223941,Varies with device,10000000,Free,0.0,Everyone,Photography,"August 6, 2018",Varies with device,Varies with device,"Recommended, 100% love it, keep good work dev ...",Positive,0.566667,0.733333
59121,A+ Gallery - Photos & Videos,PHOTOGRAPHY,4.5,223941,Varies with device,10000000,Free,0.0,Everyone,Photography,"August 6, 2018",Varies with device,Varies with device,Too much ads,Positive,0.200000,0.200000
59122,A+ Gallery - Photos & Videos,PHOTOGRAPHY,4.5,223941,Varies with device,10000000,Free,0.0,Everyone,Photography,"August 6, 2018",Varies with device,Varies with device,Just allow time ...,Neutral,0.000000,0.000000


In [13]:
merged_df['Log_Installs']=np.log(merged_df['Installs'])

In [14]:
merged_df['Reviews']=merged_df['Reviews'].astype(int)

In [15]:
merged_df['Log_Reviews']=np.log(merged_df['Reviews'])

In [16]:
merged_df['Revenue']=merged_df['Price']*apps_df['Installs']

In [17]:
import nltk
nltk.download('vader_lexicon')


[nltk_data] Error loading vader_lexicon: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [18]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()


In [19]:
merged_df['Sentiment_Score']=merged_df['Translated_Review'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

In [20]:
merged_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Current Ver,Android Ver,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Log_Installs,Log_Reviews,Revenue,Sentiment_Score
0,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,...,2.0.0,4.0.3 and up,A kid's excessive ads. The types ads allowed a...,Negative,-0.250,1.000000,13.122363,6.874198,0.0,-0.2500
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,...,2.0.0,4.0.3 and up,It bad >:(,Negative,-0.725,0.833333,13.122363,6.874198,0.0,-0.8020
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,...,2.0.0,4.0.3 and up,like,Neutral,0.000,0.000000,13.122363,6.874198,0.0,0.3612
3,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,...,2.0.0,4.0.3 and up,I love colors inspyering,Positive,0.500,0.600000,13.122363,6.874198,0.0,0.6369
4,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,...,2.0.0,4.0.3 and up,I hate,Negative,-0.800,0.900000,13.122363,6.874198,0.0,-0.5719


In [21]:

merged_df['Last Updated']=pd.to_datetime(merged_df['Last Updated'],errors='coerce')

In [22]:
merged_df['Year']=merged_df['Last Updated'].dt.year

In [23]:
html_files_path="./"
if not os.path.exists(html_files_path):
    os.mkdirs(html_files_path)

In [24]:
plot_containters = "" 

In [25]:
def save_plot_as_html(fig, filename, insight, plot_containters):
    filepath = os.path.join(html_files_path, filename)
    html_content = pio.to_html(fig, full_html=False, include_plotlyjs='inline')
    
    plot_containters += f"""
    <div class="plot-container" id="{filename}" onclick="openPlot('{filename}')">
        <div class="plot"> {html_content}</div>
        <div class="insights">{insight}</div>
    </div>
    """
    
    fig.write_html(filepath, full_html=False, include_plotlyjs='inline')
    return plot_containters

In [26]:
plot_width=400
plot_height=300
plot_bg_color='black'
text_color='white'
title_font={'size':16}
axis_font={'size':12}

In [27]:
#  This is the First Chart in the Dashboard
import pandas as pd
import plotly.express as px
import webbrowser

# Step 1: Filter apps with more than 1000 reviews
filtered_df = merged_df[merged_df['Reviews'] > 1000]

# Step 2: Keep only Top 5 categories by number of apps
top5_categories = filtered_df['Category'].value_counts().nlargest(5).index
top_df = filtered_df[filtered_df['Category'].isin(top5_categories)]

# Step 3: Create rating groups
def group_rating(rating):
    if rating <= 2:
        return "1-2"
    elif rating <= 4:
        return "3-4"
    else:
        return "4-5"

top_df['Rating Group'] = top_df['Rating'].apply(group_rating)

# Optional: Simplify category names (take only first word if needed)
top_df['Category'] = top_df['Category'].str.split().str[0]

# Step 4: Group by Category, Rating Group, and Sentiment
grouped = top_df.groupby(['Category', 'Rating Group', 'Sentiment']).size().reset_index(name='Count')

# Step 5: Pivot data
pivot_df = grouped.pivot_table(index=['Category', 'Rating Group'], 
                                columns='Sentiment', 
                                values='Count', 
                                fill_value=0).reset_index()

# Step 6: Plot small stacked bar chart
fig1 = px.bar(
    pivot_df,
    x='Rating Group',
    y=['Positive', 'Neutral', 'Negative'],
    color_discrete_map={'Positive': 'green', 'Neutral': 'gray', 'Negative': 'red'},
    facet_col='Category',
    barmode='stack',
    title='Sentiment vs Rating',
    width=600,
    height=400
)

# Step 7: Compact layout and small font styling
fig1.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(color='black', size=10),
    title_font_size=12,
    title_x=0.5,
    margin=dict(l=20, r=20, t=40, b=30)
)

# Step 8: Smaller facet titles (category)
fig1.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1], font=dict(size=10)))

# Save to HTML and open
#fig.write_html("sentiment_stacked_bar1.html")
#webbrowser.open("sentiment_stacked_bar1.html")
plot_containters = ""
plot_containters = save_plot_as_html(fig1, "Sentiment_bar Graph1.html", "The Game Category has maximum positive and negative reviews.", plot_containters)



C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12832\549057788.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_df['Rating Group'] = top_df['Rating'].apply(group_rating)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12832\549057788.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_df['Category'] = top_df['Category'].str.split().str[0]


In [28]:
df_filtered = merged_df[merged_df['Installs'] > 1_000_000]


In [29]:
df_filtered = df_filtered[~df_filtered['Category'].str.startswith(tuple("ACGS"))]


In [30]:
top5_categories = df_filtered.groupby('Category')['Installs'].sum().nlargest(5).index
df_top5 = df_filtered[df_filtered['Category'].isin(top5_categories)]


In [31]:
category_to_country = {
    'Education': 'India',
    'Finance': 'United States',
    'Health': 'Germany',
    'Entertainment': 'Canada',
    'Productivity': 'Brazil'
}

df_top5['Country'] = df_top5['Category'].map(category_to_country)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12832\1192873454.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
df_grouped = df_top5.groupby(['Country', 'Category'])['Installs'].sum().reset_index()


In [33]:
# This is the Second chart in the Dashboard
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta
import webbrowser

# Step 1: Clean 'Installs'
merged_df['Installs'] = merged_df['Installs'].astype(str).str.replace('[+,]', '', regex=True).astype(int)

# Step 2: Filter installs > 1M
df_filtered = merged_df[merged_df['Installs'] > 1_000_000]

# Step 3: Remove categories starting with A, C, G, S
df_filtered = df_filtered[~df_filtered['Category'].str.startswith(tuple("ACGS"))]

# Step 4: Top 5 categories by total installs
top5_categories = df_filtered.groupby('Category')['Installs'].sum().nlargest(5).index
df_top5 = df_filtered[df_filtered['Category'].isin(top5_categories)].copy()

# Step 5: Add random Country column
sample_countries = ['India', 'United States', 'Germany', 'Canada', 'Brazil']
np.random.seed(42)
df_top5['Country'] = np.random.choice(sample_countries, size=len(df_top5))

# Step 6: Group by Country and Category
df_grouped = df_top5.groupby(['Country', 'Category'])['Installs'].sum().reset_index()

# Step 7: Show chart only between 6 PM and 8 PM IST
ist_now = datetime.utcnow() + timedelta(hours=5, minutes=30)

if 18 <= ist_now.hour < 20:
    # Create choropleth map
    fig = px.choropleth(
        df_grouped,
        locations='Country',
        locationmode='country names',
        color='Installs',
        hover_name='Country',  # ✅ Fixed
        animation_frame='Category',  # Keep if you want category-wise animation
        color_continuous_scale='Blues',
        title='🌍 Global App Installs by Category (Top 5, >1M Installs)'
    )

    fig.update_layout(
        width=700,
        height=450,
        title_font_size=14,
        font=dict(size=10),
        margin=dict(l=30, r=30, t=40, b=30)
    )

    # Save and open
    #fig.write_html("Choropleth_Map2.html")
    #webbrowser.open("Choropleth_Map2.html")
    plot_containers = ""
    plot_containers = save_plot_as_html(fig, "Choropleth_Map2.html","installs are greater ", plot_containers)

else:
    print("⏰ Choropleth Map is hidden — visible only between 6 PM and 8 PM IST.")


⏰ Choropleth Map is hidden — visible only between 6 PM and 8 PM IST.


In [34]:
merged_df.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver', 'Translated_Review', 'Sentiment', 'Sentiment_Polarity',
       'Sentiment_Subjectivity', 'Log_Installs', 'Log_Reviews', 'Revenue',
       'Sentiment_Score', 'Year'],
      dtype='object')

In [35]:
apps_df.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')

In [36]:
merged_df.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver', 'Translated_Review', 'Sentiment', 'Sentiment_Polarity',
       'Sentiment_Subjectivity', 'Log_Installs', 'Log_Reviews', 'Revenue',
       'Sentiment_Score', 'Year'],
      dtype='object')

In [37]:
merged_df['Type']

0        Free
1        Free
2        Free
3        Free
4        Free
         ... 
59119    Free
59120    Free
59121    Free
59122    Free
59123    Free
Name: Type, Length: 59124, dtype: object

In [38]:
apps_df['Type']

0        Free
1        Free
2        Free
3        Free
4        Free
         ... 
10834    Free
10836    Free
10837    Free
10839    Free
10840    Free
Name: Type, Length: 8892, dtype: object

In [39]:
print("Original rows:", len(apps_df))

# Size in MB
apps_df = apps_df[apps_df['Size'].astype(str).str.contains('M', na=False)]
apps_df['Size'] = apps_df['Size'].astype(str).str.replace('M', '', regex=False)
apps_df['Size'] = pd.to_numeric(apps_df['Size'], errors='coerce')
print("After Size cleaning:", len(apps_df))

# Android version
apps_df['Android_Ver_Num'] = apps_df['Android Ver'].str.extract(r'(\d+\.\d+)')
apps_df['Android_Ver_Num'] = pd.to_numeric(apps_df['Android_Ver_Num'], errors='coerce')
print("After Android version:", len(apps_df))

# Revenue calculation
apps_df['Revenue'] = apps_df['Price'] * apps_df['Installs']

# Apply filters
filtered_df = apps_df[
    (apps_df['Installs'] >= 10000) &
    (apps_df['Revenue'] >= 10000) &
    (apps_df['Android_Ver_Num'] > 4.0) &
    (apps_df['Size'] > 15) &
    (apps_df['Content Rating'] == "Everyone") &
    (apps_df['App'].str.len() <= 30)
]
print("After all filters:", len(filtered_df))



Original rows: 8892
After Size cleaning: 7167
After Android version: 7167
After all filters: 31


In [40]:
print("Total rows in merged_df:", len(merged_df))

# Check Size > 10MB
size_filtered = merged_df[merged_df['Size'].astype(str).str.contains('M', na=False)]
size_filtered['Size_MB'] = pd.to_numeric(size_filtered['Size'].str.replace('M', '', regex=False), errors='coerce')
size_filtered = size_filtered[size_filtered['Size_MB'] > 10]
print("Size > 10MB:", len(size_filtered))

# Revenue calculation (if not already present)
if 'Revenue' not in merged_df.columns:
    merged_df['Revenue'] = merged_df['Price'] * merged_df['Installs']

# Revenue >= 1000
revenue_filtered = merged_df[merged_df['Revenue'] >= 1000]
print("Revenue >= 1000:", len(revenue_filtered))

# Installs >= 10000
installs_filtered = merged_df[merged_df['Installs'] >= 10000]
print("Installs >= 10000:", len(installs_filtered))

# Android version > 4.0
merged_df['Android_Ver_Num'] = pd.to_numeric(merged_df['Android Ver'].str.extract(r'(\d+\.\d+)')[0], errors='coerce')
android_filtered = merged_df[merged_df['Android_Ver_Num'] > 4.0]
print("Android version > 4.0:", len(android_filtered))

# Content Rating == Everyone
rating_filtered = merged_df[merged_df['Content Rating'] == "Everyone"]
print("Content Rating == Everyone:", len(rating_filtered))

# App name length <= 35
name_filtered = merged_df[merged_df['App'].str.len() <= 35]
print("App name length <= 35:", len(name_filtered))


Total rows in merged_df: 59124


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12832\2041083842.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Size > 10MB: 29256
Revenue >= 1000: 0
Installs >= 10000: 59050
Android version > 4.0: 27176
Content Rating == Everyone: 45213
App name length <= 35: 51044


In [41]:
# Step 1: Clean Price column if it's string like '$4.99'
merged_df['Price'] = merged_df['Price'].astype(str).str.replace('$', '', regex=False)
merged_df['Price'] = pd.to_numeric(merged_df['Price'], errors='coerce').fillna(0)

# Step 2: Clean Installs column if it contains commas or symbols
merged_df['Installs'] = merged_df['Installs'].astype(str).str.replace('[+,]', '', regex=True)
merged_df['Installs'] = pd.to_numeric(merged_df['Installs'], errors='coerce')

# Step 3: Recalculate revenue
merged_df['Revenue'] = merged_df['Price'] * merged_df['Installs']

# Step 4: Check how many pass Revenue ≥ 1000
revenue_filtered = merged_df[merged_df['Revenue'] >= 1000]
print("Fixed Revenue ≥ 1000:", len(revenue_filtered))


Fixed Revenue ≥ 1000: 519


In [42]:
#third chart
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta
import webbrowser

# Step 1: Time logic (1–2 PM IST only)
ist_now = datetime.utcnow() + timedelta(hours=5, minutes=30)
if ist_now.hour != 13:
    print(f"Current IST time: {ist_now.strftime('%H:%M')}. Chart is hidden outside 1–2 PM.")
else:
    print(f"Current IST time: {ist_now.strftime('%H:%M')}. Chart will be generated.")

    # Step 2: Clean and filter
    merged_df['Price'] = merged_df['Price'].astype(str).str.replace('$', '', regex=False)
    merged_df['Price'] = pd.to_numeric(merged_df['Price'], errors='coerce').fillna(0)

    merged_df['Installs'] = merged_df['Installs'].astype(str).str.replace('[+,]', '', regex=True)
    merged_df['Installs'] = pd.to_numeric(merged_df['Installs'], errors='coerce')

    merged_df['Revenue'] = merged_df['Price'] * merged_df['Installs']
    merged_df['Android_Ver_Num'] = pd.to_numeric(merged_df['Android Ver'].str.extract(r'(\d+\.\d+)')[0], errors='coerce')

    
    size_df = merged_df[merged_df['Size'].astype(str).str.contains('M', na=False)].copy()
    size_df['Size_MB'] = pd.to_numeric(size_df['Size'].astype(str).str.replace('M', '', regex=False), errors='coerce')


    filtered_df = size_df[
        (size_df['Installs'] >= 10000) &
        (size_df['Revenue'] >= 1000) &
        (size_df['Android_Ver_Num'] > 4.0) &
        (size_df['Size_MB'] > 10) &
        (size_df['Content Rating'] == "Everyone") &
        (size_df['App'].str.len() <= 35)
    ]

    print("Filtered rows:", len(filtered_df))

    # Step 3: Top 3 categories
    top_categories = (
        filtered_df['Category'].value_counts()
        .nlargest(3)
        .index.tolist()
    )
    top_df = filtered_df[filtered_df['Category'].isin(top_categories)]

    # Step 4: Group by Category and Type (Free/Paid)
    grouped = (
        top_df.groupby(['Category', 'Type'], as_index=False)
        .agg({'Installs': 'mean', 'Revenue': 'mean'})
    )

    # Step 5: Create Plotly dual-axis chart
    fig = go.Figure()

    for t in grouped['Type'].unique():
        df = grouped[grouped['Type'] == t]
        fig.add_trace(go.Bar(
            x=df['Category'],
            y=df['Installs'],
            name=f'Avg Installs - {t}',
            yaxis='y1'
        ))
        fig.add_trace(go.Bar(
            x=df['Category'],
            y=df['Revenue'],
            name=f'Avg Revenue - {t}',
            yaxis='y2'
        ))

    # Dual Y-axis setup
    fig.update_layout(
        title='Average Installs and Revenue by Category (Free vs Paid)',
        xaxis=dict(title='App Category'),
        yaxis=dict(title='Average Installs', side='left'),
        yaxis2=dict(title='Average Revenue', overlaying='y', side='right'),
        barmode='group',
        width=900,
        height=500
    )

    # Step 6: Save chart to HTML
    #output_file = 'dual_axis_category_chart3.html'
    #fig.write_html(output_file)
    #webbrowser.open(output_file)
    plot_containters = ""
    plot_containters = save_plot_as_html(fig, "dual_axis_category_chart3.html", "The Personlization Apps have both Avg installs and Avg Renevue.", plot_containters)




Current IST time: 17:30. Chart is hidden outside 1–2 PM.


In [43]:
# This is the Four Chart in the Dashboard
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta
import webbrowser

# Simulated data — load your actual full dataset instead
df = pd.DataFrame({
    'Category': ['GAME', 'BEAUTY', 'BUSINESS', 'DATING', 'ENTERTAINMENT', 'SOCIAL',
                 'GAME', 'SOCIAL', 'DATING', 'ENTERTAINMENT', 'BUSINESS', 'COMICS'],
    'Type': ['Paid'] * 12,
    'Installs': [150000, 80000, 60000, 40000, 50000, 30000, 90000, 70000, 65000, 62000, 61000, 55000],
    'Revenue': [50000, 10000, 20000, 12000, 18000, 15000, 30000, 17000, 16000, 14000, 19000, 12500]
})

# Simulated IST time
ist_now = datetime.utcnow() + timedelta(hours=5, minutes=30)

if 17 <= ist_now.hour < 19:
    # Relax filters to allow more data points
    filtered_df = df[
        (df['Installs'] >= 30000) &  # reduced from 50000
        (df['Category'].isin([
            'GAME', 'BEAUTY', 'BUSINESS', 'COMICS', 'COMMUNICATION',
            'DATING', 'ENTERTAINMENT', 'SOCIAL', 'EVENT'
        ]))
    ].copy()

    # Add label for coloring
    filtered_df['Category_Label'] = filtered_df['Category']

    # Highlight GAME
    color_map = {'GAME': 'pink'}

    # Scale Revenue down if too large (optional)
    filtered_df['BubbleSize'] = filtered_df['Revenue'] / 1000

    # Create bubble chart
    fig = px.scatter(
        filtered_df,
        x='Installs',
        y='Revenue',
        size='BubbleSize',
        color='Category_Label',
        hover_name='Category',
        color_discrete_map=color_map,
        title='Installs vs Revenue by Category',
        size_max=20  # reduced from 50 for smaller bubbles
    )

    fig.update_layout(
        xaxis_title='Installs',
        yaxis_title='Revenue',
        width=600,
        height=400,
        title_font_size=14,
        font=dict(size=10),
        margin=dict(l=30, r=30, t=40, b=30)
    )

    #fig.write_html("Bubble_Chart4_Adjusted.html")
    #webbrowser.open("Bubble_Chart4_Adjusted.html")
    plot_containers = ""
    plot_containers = save_plot_as_html(fig, "Bubble_Chart4_Adjusted.html", "The Game has maximum installs and revenue.", plot_containers)

else:
    print(f"Chart will not be displayed. Current time {ist_now.strftime('%H:%M')} is outside 5 PM to 7 PM IST.")


In [44]:
print("Available columns in df:", df.columns.tolist())


Available columns in df: ['Category', 'Type', 'Installs', 'Revenue']


In [45]:
#This is Fifth Chart in the Dashborad
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta
import webbrowser

# Load data (replace with your actual file path)
df = merged_df  # <-- FIX: Define df

# Simulate IST time
ist_now = datetime.utcnow() + timedelta(hours=5, minutes=30)

# Only run chart between 11 AM and 2 PM IST
if 11 <= ist_now.hour < 14:
    # Ensure 'Last Updated' is datetime
    df['Last Updated'] = pd.to_datetime(df['Last Updated'], errors='coerce')

    # Apply filters
    filtered_df = df[
        (~df['App'].str.lower().str.startswith(('x', 'y', 'z'))) &
        (~df['App'].str.contains('s', case=False, na=False)) &
        (df['Reviews'] > 500) &
        (df['Category'].str.startswith(('B', 'C', 'E')))
    ].copy()

    # Assign Month
    filtered_df['Month'] = filtered_df['Last Updated'].dt.to_period('M').dt.to_timestamp()

    # Group by Month and Category
    monthly_installs = (
        filtered_df.groupby(['Month', 'Category'])['Installs']
        .sum()
        .reset_index()
        .sort_values(['Category', 'Month'])
    )

    # Calculate MoM Growth %
    monthly_installs['MoM_Growth'] = (
        monthly_installs.groupby('Category')['Installs'].pct_change() * 100
    )

    # Base line chart
    fig5 = px.line(
        monthly_installs,
        x='Month',
        y='Installs',
        color='Category',
        title='Install Trends'
    )

    # Highlight >20% MoM growth with shaded areas
    for cat in monthly_installs['Category'].unique():
        cat_data = monthly_installs[monthly_installs['Category'] == cat]
        growth_data = cat_data[cat_data['MoM_Growth'] > 20]

        fig5.add_scatter(
            x=growth_data['Month'],
            y=growth_data['Installs'],
            mode='lines',
            name='>20% Growth',
            fill='tozeroy',
            line=dict(dash='dot', color='lightblue'),
            opacity=0.2,
            showlegend=False
        )

    # Final layout tuning
    fig5.update_layout(
        xaxis_title='Month',
        yaxis_title='Installs',
        width=600,
        height=400,
        font=dict(size=10),
        title_font_size=14,
        margin=dict(l=30, r=30, t=40, b=40)
    )

    # Define the function if not defined already
    def save_plot_as_html(fig, filename, title, plot_containers):
        fig.write_html(filename)
        webbrowser.open(filename)
        return plot_containers + f"<h2>{title}</h2><iframe src='{filename}' width='620' height='420'></iframe><br>"

    # Save and open
    plot_containers = ""
    plot_containers = save_plot_as_html(fig5, "Install_Trend5.html", "The Installing of apps increased has the nmber of years increased maximum events apps are installed.", plot_containers)

else:
    print("Chart not shown — current time is outside 11 AM to 2 PM IST.")


Chart not shown — current time is outside 11 AM to 2 PM IST.


In [50]:
import os
import webbrowser

# Path to your HTML files
html_files_path = "./"

# Chart files and their visibility times in IST (start_hour, end_hour)
chart_files = [
    "Sentiment_bar Graph1.html",      # Always visible
    "Choropleth_Map2.html",           # 6 PM – 8 PM
    "dual_axis_category_chart3.html", # 1 PM – 2 PM
    "Bubble_Chart4_Adjusted.html",    # 5 PM – 7 PM
    "Install_Trend5.html"             # 11 AM – 2 PM
]

chart_time_map = [
    (0, 24),    # Chart 1
    (18, 20),   # Chart 2
    (13, 14),   # Chart 3
    (17, 19),   # Chart 4
    (11, 14)    # Chart 5
]

plot_width = 600
plot_height = 400
plot_containers = ""

for idx, chart in enumerate(chart_files):
    chart_path = os.path.join(html_files_path, chart)
    if os.path.exists(chart_path):
        start_hour, end_hour = chart_time_map[idx]
        iframe_html = f"""
        <div class="plot-container" 
             id="chart-{idx+1}" 
             data-start="{start_hour}" 
             data-end="{end_hour}" 
             style="display: none;">
            <iframe src="{chart}" width="{plot_width}" height="{plot_height}" frameborder="0"></iframe>
            <a class="insights" href="{chart}" target="_blank">Click to expand</a>
        </div>
        """
        plot_containers += iframe_html
    else:
        print(f"Chart not found: {chart}")

dashboard_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Google Playstore Review Analytics</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #333;
            color: #fff;
            margin: 0;
            padding: 0;
        }}
        .container {{
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
            padding: 20px;
        }}
        .plot-container {{
            border: 2px solid #555;
            margin: 10px;
            padding: 5px;
            width: {plot_width}px;
            height: {plot_height + 30}px;
            position: relative;
            background-color: #222;
        }}
        .insights {{
            position: absolute;
            top: 5px;
            right: 5px;
            font-size: 12px;
            background: rgba(0,0,0,0.7);
            padding: 4px 8px;
            border-radius: 4px;
            color: white;
        }}
        .note {{
            text-align: center;
            font-size: 14px;
            margin-top: 10px;
            color: #aaa;
        }}
    </style>
    <script>
        document.addEventListener("DOMContentLoaded", function () {{
            const now = new Date();
            const utcOffset = now.getTimezoneOffset() * 60000;
            const istOffset = 5.5 * 60 * 60000;
            const istTime = new Date(now.getTime() + istOffset + utcOffset);
            const hour = istTime.getHours();
            const minutes = istTime.getMinutes();

            const charts = document.querySelectorAll(".plot-container");
            charts.forEach(chart => {{
                const start = parseInt(chart.getAttribute("data-start"));
                const end = parseInt(chart.getAttribute("data-end"));
                const chartId = chart.id;

                if (hour >= start && hour < end) {{
                    chart.style.display = "block";
                }} else {{
                    if (start === 0 && end === 24) {{
                        chart.style.display = "block";
                        return;
                    }} else {{
                        let message = `<div class='note'>Chart ${{chartId.split('-')[1]}} - Chart not shown — current time is ${{hour}}:${{minutes.toString().padStart(2, '0')}} IST. Visible only between ${{start}}:00 and ${{end}}:00 IST.</div>`;
                        chart.insertAdjacentHTML('afterend', message);
                    }}
                }}
            }});
        }});
    </script>
</head>
<body>
    <h1 style="text-align:center;">Google Playstore Review Analysis</h1>
    <div class="container">
        {plot_containers}
    </div>
</body>
</html>
"""

# Save the dashboard
dashboard_path = os.path.join(html_files_path, "web_page_dashboard2.html")
with open(dashboard_path, "w", encoding="utf-8") as f:
    f.write(dashboard_html)

# Open only when running from Python
webbrowser.open('file://' + os.path.realpath(dashboard_path))


True

In [ ]:
import http.server
import socketserver
import webbrowser
import os

PORT = 8000
FILENAME = "web_page_dashboard.html"

# Ensure the file exists
if not os.path.exists(FILENAME):
    print(f"❌ File not found: {FILENAME}")
else:
    Handler = http.server.SimpleHTTPRequestHandler
    with socketserver.TCPServer(("", PORT), Handler) as httpd:
        url = f"http://localhost:{PORT}/{FILENAME}"
        print(f"✅ Serving dashboard at: {url}")
        webbrowser.open(url)
        httpd.serve_forever()
